<a href="https://colab.research.google.com/github/awais-yaqoob-ml/ml-codes/blob/main/RNNtfSentimentAnalysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%tensorflow_version 2.x

from keras.datasets import imdb
# from keras.preprocessing import sequence
import tensorflow as tf
import numpy as np

VOCAB_SIZE = 88584
MAXLEN = 250
BATCH_SIZE = 64

(train_data, train_labels), (test_data, test_labels) = imdb.load_data(num_words=VOCAB_SIZE)

Colab only includes TensorFlow 2.x; %tensorflow_version has no effect.
17464789/17464789 [==============================] - 0s 0us/step


In [2]:
from keras.utils.data_utils import pad_sequences
## padding for maxing len equal to 250 for all if >250 trim else pad
train_data = pad_sequences(train_data, MAXLEN)
test_data = pad_sequences(test_data, MAXLEN)

In [3]:
len(train_data[66])

250

In [4]:
## model
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(VOCAB_SIZE, 32),
    tf.keras.layers.LSTM(32),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, None, 32)          2834688   
                                                                 
 lstm (LSTM)                 (None, 32)                8320      
                                                                 
 dense (Dense)               (None, 1)                 33        
                                                                 
Total params: 2,843,041
Trainable params: 2,843,041
Non-trainable params: 0
_________________________________________________________________


In [5]:
model.compile(optimizer="rmsprop",
              loss="binary_crossentropy",
              metrics=["accuracy"])
history = model.fit(train_data, train_labels, epochs=20, validation_split=0.2)

Epoch 1/20
625/625 [==============================] - 67s 95ms/step - loss: 0.4627 - accuracy: 0.7756 - val_loss: 0.3336 - val_accuracy: 0.8634
Epoch 2/20
625/625 [==============================] - 23s 37ms/step - loss: 0.2687 - accuracy: 0.8949 - val_loss: 0.3042 - val_accuracy: 0.8810
Epoch 3/20
625/625 [==============================] - 14s 23ms/step - loss: 0.2010 - accuracy: 0.9254 - val_loss: 0.3016 - val_accuracy: 0.8808
Epoch 4/20
625/625 [==============================] - 13s 21ms/step - loss: 0.1611 - accuracy: 0.9434 - val_loss: 0.2951 - val_accuracy: 0.8860
Epoch 5/20
625/625 [==============================] - 11s 18ms/step - loss: 0.1338 - accuracy: 0.9533 - val_loss: 0.3184 - val_accuracy: 0.8828
Epoch 6/20
625/625 [==============================] - 9s 14ms/step - loss: 0.1108 - accuracy: 0.9632 - val_loss: 0.3184 - val_accuracy: 0.8650
Epoch 7/20
625/625 [==============================] - 10s 16ms/step - loss: 0.0937 - accuracy: 0.9687 - val_loss: 0.5999 - val_accuracy: 

In [7]:
word_index = imdb.get_word_index()

def encode_text(text):
  tokens = tf.keras.preprocessing.text.text_to_word_sequence(text)
  tokens = [word_index[word] if word in word_index else 0 for word in tokens]
  return pad_sequences([tokens], MAXLEN)[0]

text = "the movie is good"
encode = encode_text(text)
print(encode)

[ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  1 17  6 49]


In [9]:
reverse_word_index = {value: key for (key,value) in word_index.items()}

def decode_integers(integers):
  PAD = 0
  text = ""
  for num in integers:
    if num != PAD:
      text += reverse_word_index[num]+ " "
  return text[:-1]

print(decode_integers(encode))

the movie is good


In [15]:
##predictions

def predict(text):
  encoded_text = encode_text(text)
  pred = np.zeros((1,250))
  pred[0] = encoded_text
  result = model.predict(pred)
  return result[0]

input1 = "this movie is great i will watch it again. how cool this is"
print(predict(input1))
print("------------------------------")
input2 = "this movies sucks waste of time don't waste your money"
print(predict(input2))

1/1 [==============================] - 0s 20ms/step
[0.8567798]
------------------------------
1/1 [==============================] - 0s 21ms/step
[0.7501463]
